In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from opensoundscape.datasets import SingleTargetAudioDataset
from opensoundscape.torch.train import train

/Users/tessa/Library/Caches/pypoetry/virtualenvs/opensoundscape-dxMTH98s-py3.7/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/tessa/Library/Caches/pypoetry/virtualenvs/opensoundscape-dxMTH98s-py3.7/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [3]:
from pathlib import Path
from pandas import DataFrame
from torchvision.models import resnet18
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import SGD
from math import floor
from sklearn.utils import shuffle

# Create a dataframe of `cardinalis-cardinalis` files

In [4]:
files = list(Path("/Volumes/seagate-storage/audio/og_files_from_10spp/").glob("**/*.mp3"))
labels = [file.parent.name for file in files]
df = DataFrame({'filename':files, "label":labels})
df['numeric_label'] = None
df['numeric_label'][df.label != 'cardinalis-cardinalis'] = 0
df['numeric_label'][df.label == 'cardinalis-cardinalis'] = 1

In [5]:
labels_dict = {0:'cardinalis-cardinalis', 1:'other'}
#df.drop('filename', axis=1).drop_duplicates().values

In [6]:
#data_df = df.drop('label', axis=1)

In [14]:
#train_set_size = floor(df.size * 0.8)
train_set_size = 8
val_set_size = 2
df = shuffle(df).reset_index()
train_set = df[:train_set_size]
valid_set = df[train_set_size:]

# Create a training/validation dataset
Note: this is without splitting

In [15]:
%autoreload
train_dataset = SingleTargetAudioDataset(
    df = train_set,
    label_column = 'numeric_label',
    filename_column='filename',
    tensor_augment = False, # Do spec_aug augmentations
    add_noise = True, # Add noise/jitter to spectrogram
    random_trim_length = 5, #Extract a 5s clip from long files
    overlay_prob = 1,#with a 0.2 probability, overlay files from another class
)

valid_dataset = SingleTargetAudioDataset(
    df = valid_set,
    label_column = 'numeric_label',
    filename_column='filename',
    tensor_augment = True, # Do spec_aug augmentations
    add_noise = True, # Add noise/jitter to spectrogram
    random_trim_length = 5, #Extract a 5s clip from long files
    overlay_prob = 0.2 #with a 0.2 probability, overlay files from another class
)

In [17]:
train_dataset[0]['X'].shape

torch.Size([3, 224, 224])

# Train the machine learning model

In [18]:
# Set up architecture for the type of model we will use
model = resnet18(pretrained = True)
model.fc = Linear(model.fc.in_features, 2)

In [21]:
save_path = Path('save_results')
if not save_path.exists(): save_path.mkdir()
out = train(
    save_dir = save_path,
    model = model,
    train_dataset = train_dataset,
    valid_dataset = valid_dataset,
    optimizer = SGD(model.parameters(), lr=1e-3),
    loss_fn = CrossEntropyLoss(),
    epochs=1,
    log_every=1,
)

Epoch: 0
IndexError for spectrogram of shape torch.Size([1, 224, 224])


NameError: name 'exit' is not defined